## Sampling alpha and lambda parameters
... from a multivariate normal distribution 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math


### Using the summary stats from the grid search to fit distribution

In [ ]:
google_root = "Q:"
data_path = r"\Shared drives\Pandemic Data"
model_name = "slf_model"
run_name = "slf_grid_broad" # folder of your grid search 
total_runs = 80 # Count of runs expected (this is only needed if you had variable #'s of runs - ie. from two rounds of parameter sampling)

data_dir = f"{google_root}{data_path}\{model_name}"

In [ ]:
stats_dir = f"{data_dir}/outputs/summary_stats/{run_name}"
# input_dir = "inputs"
input_dir = f"{data_dir}/inputs/noTWN"

In [ ]:
stats = pd.read_csv(f"{stats_dir}/summary_stats_wPrecisionRecallF1FBetaAggProb.csv")
stats = stats.groupby("sample").filter(lambda x: len(x) == total_runs)

In [ ]:
## Aggregating needed stats by sample

agg_dict = {
    "start":["max"],
    "alpha":["max"],
    "lamda": ["max"],
    "count_known_countries_time_window_fbeta": ["mean","std"]
}

agg_dict = {**agg_dict}

agg_df = stats.groupby("sample").agg(agg_dict)

agg_df.columns = ["_".join(x) for x in agg_df.columns.values]

In [ ]:
agg_df = agg_df.rename(columns={"start_max":"start","alpha_max":"alpha","lamda_max":"lamda","count_known_countries_time_window_fbeta_mean":"fbeta"})
agg_df['st_err']=agg_df['count_known_countries_time_window_fbeta_std']/np.sqrt(50)

### Exploring possible quantile thresholds for fbeta

In [ ]:
count_vals = []
min_fbeta = []

for val in range(70,100):
    subset = agg_df.loc[agg_df['fbeta']>=agg_df['fbeta'].quantile(val/100)]
    count_vals.append(len(subset.index))
    min_fbeta.append(subset['fbeta'].min())

In [ ]:
sample_stats = pd.DataFrame({"quantile":range(70,100), "count":count_vals, "min_fbeta":min_fbeta}).set_index("quantile")

In [ ]:
quant_threshold = 90

How many samples and what Fbeta scores are captured with each threshold?

In [ ]:
fig, (ax1, ax2, ) = plt.subplots(1, 2, figsize=(10, 5))
sample_stats["count"].plot(ax = ax1)
ax1.vlines(quant_threshold, ymin=sample_stats["count"].min(), ymax=sample_stats["count"].max(), linestyle='dashed', color="firebrick")
ax1.set_title("Count")

sample_stats["min_fbeta"].plot(ax = ax2)
ax2.vlines(quant_threshold, ymin=sample_stats["min_fbeta"].min(), ymax=sample_stats["min_fbeta"].max(), linestyle='dashed', color="firebrick")
ax2.set_title("Fbeta")

plt.show()

What do the distributions of alpha and lamda look like with that threshold?

In [ ]:
agg_df['top']=np.where(agg_df['fbeta']>=agg_df['fbeta'].quantile(quant_threshold/100),'top','low')

In [ ]:
## The full faceted plot - but it is slow to run (and long)

# ax = sns.relplot(x="lamda",y="fbeta", row="alpha",hue="top",palette="rocket",
#             col="start",data=agg_df,edgecolor="black",linewidth=0.5,s=100)
# plt.show()

In [ ]:
# Alpha by year

ax = sns.relplot(x="alpha",y="fbeta", col="start",hue="top",palette="rocket",data=agg_df,edgecolor="black",linewidth=0.5,s=100)
plt.show()

In [ ]:
# Lamda by year

ax = sns.relplot(x="lamda",y="fbeta", col="start",hue="top",palette="rocket",data=agg_df,edgecolor="black",linewidth=0.5,s=100)
plt.show()

In [ ]:
# Top parameter distribution plot

ax = sns.relplot(x="alpha", y="lamda", col="start", hue="fbeta", palette="mako_r", data=agg_df.loc[agg_df['top']=="top"])
plt.show()

### Generating the multivariate normal distribution and sampled parameters

In [ ]:
# Fits a separate distribution per year 

param_samples_df = pd.DataFrame(columns=['alpha','lamda','start'])
n = 500

for year in [2005,2006,2007]:
    top_sets=agg_df.loc[((agg_df['top']=="mid") | (agg_df['top']=="top")) & (agg_df['start'] == year)][["alpha","lamda","fbeta"]]
    param_mean = np.mean(top_sets[["alpha","lamda"]].values, axis=0)
    param_cov = np.cov(top_sets[["alpha","lamda"]].values, rowvar=0)
    param_sample = np.random.multivariate_normal(param_mean, param_cov, n)
    alpha = param_sample[:,0]
    lamda = param_sample[:,1]
    start = [year]*n
    param_sample_df = pd.DataFrame({"alpha":alpha, "lamda":lamda, "start":start})
    param_samples_df = pd.concat([param_samples_df, param_sample_df])

    print(f"Year: {year}, Means: {param_mean}, Covariance Matrix: {param_cov}")
    

param_samples_df = param_samples_df.loc[param_samples_df['alpha']<=1]

In [ ]:
# Plot to visually examine - should show similar patterns to top parameter distribution plot above

ax = sns.relplot(x="alpha", y="lamda", col="start", data=param_samples_df)
plt.show()

## Writing out 1,000 sampled parameters to runs
Use this to replace the content of the commands.txt text file on HPC

In [ ]:
total = len(agg_df.loc[((agg_df['top']=="mid") | (agg_df['top']=="top"))])
count2005 = len(agg_df.loc[((agg_df['top']=="mid") | (agg_df['top']=="top")) & (agg_df['start'] == 2005)])
count2006 = len(agg_df.loc[((agg_df['top']=="mid") | (agg_df['top']=="top")) & (agg_df['start'] == 2006)])
count2007 = len(agg_df.loc[((agg_df['top']=="mid") | (agg_df['top']=="top")) & (agg_df['start'] == 2007)])

In [ ]:
round2005 = round(count2005/total * 1000)
round2006 = round(count2006/total * 1000)
round2007 = round(count2007/total * 1000)

In [ ]:
round2005 + round2006 + round2007 == 1000

In [ ]:
samples2005 = param_samples_df.loc[param_samples_df['start']==2005].reset_index(drop=True)[0:round2005]
samples2006 = param_samples_df.loc[param_samples_df['start']==2006].reset_index(drop=True)[0:round2006]
samples2007 = param_samples_df.loc[param_samples_df['start']==2007].reset_index(drop=True)[0:round2007]

In [ ]:
samples1000 = pd.concat([samples2005,samples2006,samples2007]).reset_index()

In [ ]:
# 1000 random samples

commands = ""

start_run = 0 
end_run = 0

script = "python model_run_args.py"

for i in samples1000.index:
    commands += (
        " ".join(
            [
                script,
                str(round(samples1000['alpha'][i],4)),
                str(round(samples1000['lamda'][i],4)),
                str(samples1000['start'][i]),
                str(start_run),
                str(end_run),
            ]
        )
        + "\n"
    )
print(commands)

### Creating the aggregated stats of the sampled params
After you run the model with the sampled parameters, to generate the overall summary statistics (across samples)

In [ ]:
run_name = "slf_sampled_params" # Set this to the name of the runs you ran with the sampled parameters
stats_dir = f"{data_dir}/outputs/summary_stats/{run_name}"
# input_dir = "inputs"
input_dir = f"{data_dir}/inputs/noTWN"

In [ ]:
stats = pd.read_csv(f"{stats_dir}/summary_stats_wPrecisionRecallF1FBetaAggProb.csv")

In [ ]:
# Code from summary_stats.py to define variables

sim_years = [2014,2020]
coi = "USA"

year_probs_dict_keys = []
for year in sim_years:
    year_probs_dict_keys.append(f"prob_by_{year}_{coi}")

validation_df = pd.read_csv(
        input_dir + "/first_records_validation.csv", header=0, index_col=0,
    )

countries_dict_keys = []
for ISO3 in validation_df.index:
    countries_dict_keys.append(f"diff_obs_pred_metric_{ISO3}")

In [ ]:
# Code from summmary_stats.py to define functions

def mse(x):
    return sum(x) / len(x)


def avg_std(x):
    """
    Compute average standard deviation when aggregating across runs
    of a parameter sample
    """
    return math.sqrt(sum(x ** 2) / len(x))


In [ ]:
data = stats.copy()
summary_stat_path = stats_dir

In [ ]:
# Code from get_stats.py to aggregate summary stats

agg_dict = {
    "start": ["max"],
    "alpha": ["max"],
    "lamda": ["max"],
    "total_countries_intros_predicted": ["mean", "std"],
    "diff_total_countries": ["mean", "std"],
    "diff_total_countries_sqrd": [mse],
    "count_known_countries_time_window": ["mean", "std"],
    "diff_obs_pred_metric_mean": ["mean"],
    "diff_obs_pred_metric_stdev": [avg_std],
    "count_known_countries_time_window_recall": ["mean"],
    "count_known_countries_time_window_precision": ["mean"],
    "count_known_countries_time_window_f1": ["mean"],
    "count_known_countries_time_window_fbeta": ["mean"],
}
prob_agg_dict = dict(
    zip(year_probs_dict_keys, ["mean" for i in range(len(year_probs_dict_keys))])
)
countries_agg_dict = dict(
    zip(
        countries_dict_keys,
        [["mean", "std"] for i in range(len(countries_dict_keys))],
    )
)

agg_dict = {**agg_dict, **prob_agg_dict, **countries_agg_dict}

agg_df = data.groupby('run_num').agg(agg_dict)

agg_df.columns = ["_".join(x) for x in agg_df.columns.values]
agg_df.to_csv(summary_stat_path + "/summary_stats_overall.csv")